<a href="https://colab.research.google.com/github/Arup3201/Summarization-Project-using-Pointer-Gen/blob/main/Get_To_The_Point_Summarization_with_Pointer_Generator_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pathlib
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
path_to_cnn_stories = tf.keras.utils.get_file(
    origin="https://huggingface.co/datasets/cnn_dailymail/resolve/main/data/cnn_stories.tgz",
    extract=True
)

path_to_dailymail_stories = tf.keras.utils.get_file(
    origin="https://huggingface.co/datasets/cnn_dailymail/resolve/main/data/dailymail_stories.tgz",
    extract=True
)

375893739/375893739 [==============================] - 7s 0us/step


In [4]:
path_to_cnn_stories, path_to_dailymail_stories

('/root/.keras/datasets/cnn_stories.tgz',
 '/root/.keras/datasets/dailymail_stories.tgz')

In [5]:
!ls -l /root/.keras/datasets

total 521964
drwxr-xr-x 3 root root      4096 Jul 25 16:17 cnn
-rw-r--r-- 1 root root 158577824 Jul 25 16:17 cnn_stories.tgz
drwxr-xr-x 3 root root      4096 Jul 25 16:17 dailymail
-rw-r--r-- 1 root root 375893739 Jul 25 16:17 dailymail_stories.tgz


In [6]:
cnn_stories_dir = pathlib.Path('/root/.keras/datasets/cnn/stories')
dailymail_stories_dir = pathlib.Path('/root/.keras/datasets/dailymail/stories')

In [7]:
cnn_stories_dir, dailymail_stories_dir

(PosixPath('/root/.keras/datasets/cnn/stories'),
 PosixPath('/root/.keras/datasets/dailymail/stories'))

In [8]:
def print_filenames(dir_path, num_files=5):
  '''Prints the name of the files that are present at `dir_path`.
  Maximum `num_files` number of files are shown.

  Arguements:
    dir_path: PosixPath, pointing to the directory of which the user
              wants to prints the file names.
    num_files: int, number of files user wants to print.

  returns:
    nothing
  '''

  count = 0
  for f in dir_path.glob('*.story'):
    print(f.name)
    count += 1

    if count == num_files:
      break
  else:
    print(f"Less than {num_files} is present!")

In [9]:
print_filenames(cnn_stories_dir)

18e62a1fc7bc82e6462ba17c60f483edc81b3d16.story
4a0b12b0be01248d390e00159fa0d14a396a9e9a.story
f57e2aab804726fba2027b726c2063acd6eb4249.story
0a50865864a332a56d91e2929d8a5cf7a185107d.story
060a3ac3ed2f8ab619d393e46ec916da6f420113.story


In [10]:
print_filenames(dailymail_stories_dir)

5c30a6845875a373a821ba85d3453e87cefd897d.story
571e18e2e4b0f659b10507afac3a65af7160cf50.story
49795c14f44fbdffcbf9498a821193d023c3348e.story
4ebd9c69081672d213ed77c043fd55e42a5baeb5.story
63d35ff8ff6820121cb6517f3d001529b3c8734a.story


In [32]:
# Define the global variables
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"',
              dm_single_close_quote, dm_double_close_quote, ")"]

# Maximum stories to process from cnn and dailymail each
MAX_STORIES = 50000

# From the total data how to split into train, val and test
TRAIN_SIZE = 0.8
VAL_SIZE = 0.1
TEST_SIZE = 0.1

# For tokenization
VOCAB_SIZE = 20000
OOV_TOKEN = "<UNK>"

In [12]:
# Taking a sample .story file from cnn stories
sample_filename = "438411e10e1ef79b47cc48cd95296d85798c1e38.story"
sample_filedir = cnn_stories_dir

sample_filepath = sample_filedir / sample_filename
with open(sample_filepath, 'r') as f:
  sample_story = f.read()

print(sample_story)

New York (CNN) -- The U.S. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on New Year's Eve, according to new census data released Thursday.

The figure represents a 0.7% increase from last year, adding 2,250,129 people to the U.S. population since the start of 2011, and a 1.3% increase since Census Day, April 1, 2010.

The agency estimates that beginning in January, one American will be born every eight seconds and one will die every 12 seconds.

U.S.-bound immigrants are also expected to add one person every 46 seconds.

That combination of births, deaths and migration is expected to add a single person to the U.S. population every 17 seconds, the Census Bureau said.

Meanwhile, millions are set to ring in the new year.

In New York, authorities are preparing for large crowds in Manhattan's Times Square, where Lady Gaga is expected to join Mayor Michael Bloomberg to push the button that drops the Waterford 

I am creating a function `fix_missing_period` where I am taking 2 arguements, one for the `line` for which I am checking and fixing the period and other is `end_tokens` which is a list that has all the tokens that I should consider as ending of a sentence.

These are the steps -
1. Check if line contains `@highlight`, if True then just return the line.
2. Check if line is empty, then return line as it is.
3. Check is line ends with any of the `end_tokens`, if so then return line as it is.
4. Only is none of the above conditions match then append `.` to the current line.

In [13]:
def fix_missing_period(line, end_tokens=END_TOKENS):
  '''function to fix the missing periods for some story lines which do not end with
  any of the end_tokens mentioned.

  Arguements:
    line: string, line of the story to fix the missing the period of.
    end_tokens: list of strings, all the tokens that are considered as line end.

  Returns:
    new line with fixed the ending part by adding an ending token if not present.
  '''
  if "@highlight" in line:
    return line
  elif line == "":
    return line
  elif line[-1] in end_tokens:
    return line

  return line + '.'

In [14]:
fix_missing_period(sample_story.split('\n')[0])

"New York (CNN) -- The U.S. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on New Year's Eve, according to new census data released Thursday."

I am creating a function `split_article_summary` which will split the story into article and summary parts.

The function takes only 1 arguement and that is the `story` which will be splitted into article and summary.

The steps to follow are -
1. Split the story by new line `\n`. I will get a list of lines.
2. Strip the lines by using list comprehension.
3. Use list comprehension to make lower case each line by using `.lower()`.
4. Fix each line by adding period if there is none in that line using `fix_missing_period` function.
5. Make 2 empty list for `article` and `summary`.
6. Go through each line. In each line, I need to check 4 things,
  * line contains `@highlight` or not, if True then set `next_highlight` to `True` because the next to next line is going to be a summary line.
  * line is `""` empty or not, if True then ignore.
  * `next_highlight` is True or not, if True then append the line to `summary`.
  * If non of the ebove then append to `article`.
7. After done with filling the `article` and `summary` list with lines, join those sentences to make the whole article and summary. Here, I am using `.join()` method.

In [15]:
def split_article_summary(story):
  '''Splits the story into 2 parts, one for article and other for summary of that
  article. Returns the article and summary.

  Arguements:
    story: string file that contains both article and summary combiningly.

  Returns:
    article, summary seperately from the story.

  '''
  lines = story.split('\n')
  lines = [line.strip() for line in lines]
  lines = [line.lower() for line in lines]

  # Fix the ending period
  lines = [fix_missing_period(line) for line in lines]

  # List to contain the article and summary lines
  article = []
  summary = []

  # Indicator of whether the next line is the summary or not
  next_highlight = False

  for line in lines:
    if "@highlight" in line:
      next_highlight = True
    elif line=="":
      continue
    elif next_highlight:
      summary.append(line)
    else:
      article.append(line)

  article = ' '.join(article)
  summary = ' '.join(summary)

  return article, summary

In [16]:
split_article_summary(sample_story)

('new york (cnn) -- the u.s. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on new year\'s eve, according to new census data released thursday. the figure represents a 0.7% increase from last year, adding 2,250,129 people to the u.s. population since the start of 2011, and a 1.3% increase since census day, april 1, 2010. the agency estimates that beginning in january, one american will be born every eight seconds and one will die every 12 seconds. u.s.-bound immigrants are also expected to add one person every 46 seconds. that combination of births, deaths and migration is expected to add a single person to the u.s. population every 17 seconds, the census bureau said. meanwhile, millions are set to ring in the new year. in new york, authorities are preparing for large crowds in manhattan\'s times square, where lady gaga is expected to join mayor michael bloomberg to push the button that drops the waterford cr

I am creating a function `get_articles_summaries` which will process each of the stories present in the directory of cnn and dailymail and return the articles, summaries in the form of list.

This function will take 2 arguements. One will be the `stories_dir` which is a Posix format string from `pathlib` library and another arguement is of `max_stories` which is the maximum number of stories that we will extract from those directories.

The process is simple. We will follow this steps -
1. Create 2 empty lists of `articles` and `summaries`.
2. Loop through all the files present in the directory `stories_dir` using `.glob` generator method.
3. Make a `count` variable which will count the number of processed strories and when it hits `max_stories`, break from the loop.
4. Inside the loop, you will open the file in `r` reading format, then just use `.read()` method to read the story.
5. Everytime after reading the story, split the article and summary part from it and then append them inside the `articles` and `summaries` list.
6. Return the 2 lists.

In [17]:
def get_articles_summaries(stories_dir, max_stories):
  '''stores the stories from stories_dir folder into a list and returns the list

  Arguement:
    stories_dir: Posix string, the directory where the stories are stored
    max_stories: maximum number of stories to store

  Returns:
    list of stories.

  '''
  articles = []
  summaries = []

  count = 0
  for f in stories_dir.glob("*.story"):
    count += 1
    with open(f, 'r') as reader:
      story = reader.read()

      article, summary = split_article_summary(story)

      articles.append(article)
      summaries.append(summary)

    if count == max_stories:
      break

  return articles, summaries

```
cnn
  stories
    438411e10e1ef79b47cc48cd95296d85798c1e38.story
    e453e379e8a70af2d3dff1c75c41b0a35edbe9cc.story
    2079f35aca44978a7985afe0ddacdf02bedf98f2.story
    4702f28c198223157bb8f69665b039d560eebb0f.story
    db3e2ea79323a98379228b17cd3b9dec17dbd2cb.story
    ...
    ...
    ...

dailymail
  stories
    f4ba18635997139c751311b9f2ad18f455dd7c98.story
    4a3ef32cff589c85ad0d22724e2ed747c0dacf87.story
    5375ed75939108c72001b043d3b4799c47f32be9.story
    fe9e57c21e21fb4ec26e394f0e92824f38d18a95.story
    6a544b5cdd2384be6cc657b265d7aa2de72a99e0.story
    ...
    ...
    ...

```

Out of all available .story files, we will only take `MAX_STORIES` number of files and then open them.

In [18]:
cnn_articles, cnn_summaries = get_articles_summaries(cnn_stories_dir, MAX_STORIES)

len(cnn_articles)

50000

In [19]:
print(f"Total no of cnn stories captured are {len(cnn_articles)}\n\n")
print(f"One of the CNN articles: {cnn_articles[0]}\n\n")
print(f"The summary of this article: {cnn_summaries[0]}\n\n")

Total no of cnn stories captured are 50000


One of the CNN articles: (tribune media services) -- we don't know where to look first. the massive pillars, looking like tree trunks, stone chameleons, tortoises and turtles, help support the columns. the sheer size of the place is amazing. some of the towers soar more than 500 feet. even jaded teens, like my 13-year-old niece, erica fieldman, can't help but be impressed. antoni gaudi's sagrada familia is barcelona's most famous site and spain's most visited. welcome to antoni gaudi's unfinished masterpiece, the sagrada familia, barcelona's most famous site and spain's most visited. more than 40 years after the eccentric and revered architect's death -- he was struck by a tram -- work still continues on the huge church first begun in 1882. some 2.5 million people visited last year. this is a great place to engage the kids in a scavenger hunt. (find the fruit carved on top of the towers, the young stone musicians, the birds.) there are two c

In [20]:
dailymail_articles, dailymail_summaries = get_articles_summaries(dailymail_stories_dir,
                                                                 MAX_STORIES)

In [21]:
print(f"Total no of cnn stories captured are {len(dailymail_articles)}\n\n")
print(f"One of the CNN articles: {dailymail_articles[0]}\n\n")
print(f"The summary of this article: {dailymail_summaries[0]}\n\n")

Total no of cnn stories captured are 50000


One of the CNN articles: by. ian sparks. published:. 08:22 est, 16 january 2013. |. updated:. 12:17 est, 16 january 2013. prince albert of monaco and his sisters have branded a new hollywood film about the life of their mother grace kelly as 'pointlessly glamorised and historically inaccurate'. the ruling family of the tiny mediterranean tax haven also attacked some scenes in the forthcoming movie starring nicole kidman as 'pure fiction'. american-born kelly was one of the world's most famous film stars until she married albert's father in 1956 and became princess grace of monaco. anger: prince albert of monaco and his sisters princess caroline (left) and princess stephanie (right) have branded a film about their mother grace kelly as 'pointlessly glamorised' the high society and dial m for murder actress died in a car accident on a winding road above the principality in 1982 at the age of 52. oscar-winner kidman is now starring as kelly in 

In [22]:
[1, 2] + [3, 4]

[1, 2, 3, 4]

In [23]:
random.seed(0) # Keeps the shuffling same as before
random.sample([1, 2, 3, 4, 5], 5)

[4, 5, 1, 2, 3]

In [24]:
sample_article, sample_summary = split_article_summary(sample_story)

In [25]:
sample_article

'new york (cnn) -- the u.s. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on new year\'s eve, according to new census data released thursday. the figure represents a 0.7% increase from last year, adding 2,250,129 people to the u.s. population since the start of 2011, and a 1.3% increase since census day, april 1, 2010. the agency estimates that beginning in january, one american will be born every eight seconds and one will die every 12 seconds. u.s.-bound immigrants are also expected to add one person every 46 seconds. that combination of births, deaths and migration is expected to add a single person to the u.s. population every 17 seconds, the census bureau said. meanwhile, millions are set to ring in the new year. in new york, authorities are preparing for large crowds in manhattan\'s times square, where lady gaga is expected to join mayor michael bloomberg to push the button that drops the waterford cry

I am creating another function -
`split_dataset(train_size, val_size, test_size)`: I am creating this function to split the original 1,00,000 examples into 80,000 training samples, 10,000 val samples and 10,000 test samples.

In [28]:
def split_dataset(dataset, train_size, val_size, test_size):
  first_split = train_size
  second_split = train_size+val_size
  third_split = train_size+val_size+test_size
  return dataset[:first_split, :], dataset[first_split:second_split, :], dataset[second_split:third_split, :]

Utilize the 4 functions created above into one function called `make_datasets`. This function will -
1. This functions will have many argumenets and among them 2 argumenets `cnn_stories` and `dailymail_stories` are lists which has list of articles and summaries at 0 and 1 index. It means `cnn_stories[0]` is articles of cnn news and `cnn_stories[1]` is summaries of cnn news. It applies to `dailymail_stories` as well.
Objective of this step is to concatenate the cnn articles with dailymail articles and cnn summaries with dailymail summaries.
```python
[1, 2] + [3, 4] = [1, 2, 3, 4]
```

3. Convert the articles and summaries list into tensors and then concatenate them along a new axis. To create new axis I can use `tf.newaxis` in the indexing. E.g.
```python
  np.concatenate([articles[:, tf.newaxis], summaries[:, tf.newaxis]], axis=-1)
```
4. Shuffle the dataset using `random.sample` method.
```python
random.seed(seed_value) # To make sure that everytime it gives the same shuffle
random.sample(list_to_shuffle, len(list_to_shuffle))
```
5. Split the dataset into 3 parts, one for training, other for validation and last one for testing. All the tensors are of shape `(num_samples, 2)`.

In [29]:
def make_datasets(cnn_stories, dailymail_stories, train_fraction, val_fraction, test_fraction, seed_value=0):
  '''Create 3 datasets each for training, validation and testing respectively.
  This function concatenates the articles, summaries of cnn and dailymail news. After that it will tokenize
  them one by one in a loop. After it is done with the tokenization, it will shuffle the articles and
  summaries using random.sample method (although we have a helper function for it). Finally we do the
  splitting of the whole dataset. Remember here the returned values become tensors.

  Arguements:
    cnn_stories: list of 2 values, one for cnn articles and other for cnn summaries.
    dailymail_stories: list of 2 values, one for dailymail articles and other for dailymail summaries.
    train_size: float, specifying how much fraction of the original dataset to take for training.
    val_size: float, specifying how much fraction of the original dataset to take for validation.
    test_size: float, specifying how much fraction of the original dataset to take for testing.

  Returns:
    returns a tuple with 3 values inside it, `training_data`, `validation_data` and `testing_data`
    with the specified amount of data in it.
    Each one of them are tensor with shape `(num_samples, 2)`. `shape[1]=2` for article and summary.
  '''
  articles = cnn_stories[0] + dailymail_stories[0]
  summaries = cnn_stories[1] + dailymail_stories[1]

  articles = np.array(articles, dtype=object)
  summaries = np.array(summaries, dtype=object)

  dataset = np.concatenate((articles[:, tf.newaxis], summaries[:, tf.newaxis]), axis=-1)

  random.seed(seed_value)
  shuffled_indices = random.sample(list(range(dataset.shape[0])), dataset.shape[0])

  dataset = dataset[shuffled_indices, :]

  train_size = int(train_fraction * dataset.shape[0])
  val_size = int(val_fraction * dataset.shape[0])
  test_size = dataset.shape[0] - (train_size + val_size)

  training_samples, validation_samples, testing_samples = split_dataset(dataset,
                                                                        train_size,
                                                                        val_size,
                                                                        test_size)

  return (training_samples, validation_samples, testing_samples)

In [30]:
train_dataset, val_dataset, test_dataset = make_datasets([cnn_articles, cnn_summaries], [dailymail_articles, dailymail_summaries], TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

In [31]:
print(f"Type of the datasets: {type(train_dataset)}\n")

print(f"Training dataset shape: {train_dataset.shape}")
print(f"Validation dataset shape: {val_dataset.shape}")
print(f"Testing dataset shape: {test_dataset.shape}\n")

print(f"First example in the training dataset looks like: \n {train_dataset[0]}\n")

Type of the datasets: <class 'numpy.ndarray'>

Training dataset shape: (80000, 2)
Validation dataset shape: (10000, 2)
Testing dataset shape: (10000, 2)

First example in the training dataset looks like: 
 ["by. daily mail reporter and associated press reporter. published:. 01:54 est, 1 december 2012. |. updated:. 01:54 est, 1 december 2012. a kindergarten teacher locked a 5-year-old boy in a small, dark room alone at the end of a school day then forgot the child was there for over an hour, the boy’s outraged father says. school officials in caldwell, idaho, said friday they were investigating after hearing from james cagle, who says that when his wife found their son he was crying and had urinated on himself. ‘he was scared,’ cagle told boise television station ktvb. ‘you know just think about that… a 5-year-old boy.’ scroll down for video. furious father: james cagle says he is outraged that a kindergarten teacher locked a 5-year-old boy in a small, dark room alone at the end of a sc

Now, I need to find the tokens from the articles. I need to use only training articles not any other and also I will not use summaries data because that will be my target and I won't know what type of words I will encounter when summarizing the source article. So, the only words that I know will be from the articles of training dataset. Here, I am going to use the `tensorflow.keras.preprocessing.text.Tokenizer` in short `Tokenizer` to find the tokens from the articles and then finally converting the articles into sequence of integers. One thing to remember is here we are going to use `oov_token` arguement of `Tokenizer` to mention the token we want to use for out-of-vocabulary words.

In [61]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE,
                      filters='#*+-/:,;<=>@[\\]^_{|}~\t\n[0-9]',
                      oov_token=OOV_TOKEN)

In [62]:
# Fit the tokenizer on articles of the training set
tokenizer.fit_on_texts(list(train_dataset[:, 0]))

In [63]:
print(f"The vocabulary for the tokenizer has a length {len(tokenizer.word_index)}")
print(f"'<UNK>' word had index: {tokenizer.word_index['<UNK>']}")
print(f"'teacher' word has index: {tokenizer.word_index['teacher']}")

print(f"Text to Sequence of the first article is {tokenizer.texts_to_sequences([train_dataset[0, 0]])}")

sample_sequence = tokenizer.texts_to_sequences([train_dataset[0, 0]])
print(f"Sequence to Text of the first acrticle is {tokenizer.sequences_to_texts(sample_sequence)}")

The vocabulary for the tokenizer has a length 760011
'<UNK>' word had index: 1
'teacher' word has index: 1934
Text to Sequence of the first article is [[237, 499, 578, 2951, 6, 1519, 685, 1814, 296, 74, 42, 3649, 215, 42, 599, 29, 482, 74, 412, 74, 42, 3649, 215, 42, 599, 29, 482, 4, 17812, 1934, 2991, 4, 146, 63, 95, 786, 7, 4, 6974, 1803, 640, 1813, 17, 2, 229, 5, 4, 213, 133, 111, 11314, 2, 407, 10, 70, 9, 71, 27, 7416, 2, 1, 8104, 385, 549, 213, 233, 7, 1, 19494, 21, 492, 31, 38, 1931, 40, 905, 20, 700, 1, 33, 134, 8, 44, 15, 391, 121, 37, 436, 13, 10, 4867, 6, 36, 1, 11, 3677, 2800, 10, 1, 1, 67, 1, 1125, 1167, 1, 4823, 158, 64, 137, 46, 1, 4, 146, 63, 95, 1, 440, 103, 9, 477, 5174, 385, 700, 1, 134, 13, 12, 8104, 8, 4, 17812, 1934, 2991, 4, 146, 63, 95, 786, 7, 4, 6974, 1803, 640, 1813, 17, 2, 229, 5, 4, 213, 133, 6, 111, 11314, 13, 10, 70, 9, 71, 27, 6244, 1512, 80, 1, 134, 8, 44, 15, 391, 121, 37, 436, 13, 10, 4867, 6, 36, 1, 11, 3677, 1, 21, 13, 6, 15, 391, 14783, 44, 37, 436,

19494